In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [2]:
# %pip install python-docx datasets ragas
# %pip install datasets

In [ ]:
"""
RAGAS 평가 스크립트 (rag_module.py + RAGAS 학습용 질문.docx)

입력 파일:
- /chatbot_app/chatbot_app/modules/rag_module.py
- /data/RAGAS 학습용 질문.docx

주의:
- 환경변수 필요: PINECONE_API_KEY, UPSTAGE_API_KEY, OPENAI_API_KEY
- Pinecone 인덱스는 rag_module.py의 INDEX_NAMES 설정(law-index/rule-index/case-index)과 일치해야 함.
"""

import os
import re
import sys
from typing import List, Dict, Any

from dotenv import load_dotenv
load_dotenv()   


# 1) 의존성
from docx import Document as DocxDocument
from datasets import Dataset

# RAGAS (버전에 따라 import 경로가 다를 수 있어 try 처리)
try:
    from ragas import evaluate
    from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
except Exception as e:
    raise ImportError(
        "ragas import 실패. `pip install -U ragas` 후 다시 시도하세요."
    ) from e

# 2) 팀원 RAG 모듈 import
#    파일이 /mnt/data에 있으니 sys.path에 추가
MODULES_PATH = os.path.join(os.getcwd(), "chatbot_app", "modules")
sys.path.append(MODULES_PATH)
sys.path.append(os.getcwd())
from chatbot_app.modules.rag_module import create_pipeline, RAGConfig


DOC_PATH = "./data/RAGAS 학습용 질문.docx"


def parse_ragas_docx(doc_path: str) -> List[Dict[str, Any]]:
    """
    문서에서 '번호. 질문 ... ✔️ 모범답안 ...' 패턴을 파싱해
    [{"question": ..., "ground_truths": [...]}] 로 반환.

    - 질문은 해당 번호 블록에서 '✔️ 모범답안' 이전까지 전체 텍스트
    - 정답은 '✔️ 모범답안' 이후 다음 번호 시작 전까지 전체 텍스트
    """
    doc = DocxDocument(doc_path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
    text = "\n".join(paras)

    # 번호 블록 split: "1. ..." "2. ..." (줄 시작 기준)
    blocks = re.split(r"\n(?=\d+\.\s)", text)

    items = []
    for b in blocks:
        b = b.strip()
        if not re.match(r"^\d+\.\s", b):
            continue

        # ✔️ 모범답안 기준 분리
        parts = b.split("✔️ 모범답안", 1)
        if len(parts) != 2:
            # 모범답안 없는 블록은 스킵 (문서 형식이 깨졌을 가능성)
            continue

        q_part = re.sub(r"^\d+\.\s*", "", parts[0].strip())
        a_part = parts[1].strip()

        # 모범답안 앞부분에 (법조문 힌트/포인트 등)가 섞여 있어도 질문에 포함
        question = q_part.strip()
        ground_truth = a_part.strip()

        items.append(
            {
                "question": question,
                "ground_truths": [ground_truth],
            }
        )

    # sanity check: 29개 기대
    if len(items) == 0:
        raise ValueError("docx에서 문항을 1개도 파싱하지 못했습니다. 문서 포맷을 확인하세요.")
    return items


def build_eval_dataset(pipeline, items: List[Dict[str, Any]], *, skip_normalization: bool = False) -> Dataset:
    """
    각 질문을 pipeline.answer_with_trace로 실행하여
    RAGAS가 요구하는 dataset 필드(question, answer, contexts, ground_truths)를 구성.
    """
    rows = []
    for i, ex in enumerate(items, start=1):
        q = ex["question"]

        trace = pipeline.answer_with_trace(q, skip_normalization=skip_normalization)
        answer = trace.get("answer", "") or ""
        docs = trace.get("docs", []) or []

        # contexts: List[str] (retrieved chunk 텍스트)
        contexts = []
        for d in docs:
            try:
                contexts.append(d.page_content)
            except Exception:
                pass

        rows.append(
            {
                "id": i,
                "question": q,
                "answer": answer,
                "contexts": contexts,
                "ground_truths": ex["ground_truths"],
                # 디버깅용(원하면 유지)
                "normalized_query": trace.get("normalized_query", ""),
                "reference": ex["ground_truths"][0],
            }
        )

    return Dataset.from_list(rows)


def main():
    # 0) 환경변수 체크
    required_env = ["PINECONE_API_KEY", "UPSTAGE_API_KEY", "OPENAI_API_KEY"]
    missing = [k for k in required_env if not os.getenv(k)]
    if missing:
        raise EnvironmentError(f"필수 환경변수가 없습니다: {missing}")

    # 1) 질문/정답(모범답안) 로드
    items = parse_ragas_docx(DOC_PATH)
    print(f"✅ 파싱된 문항 수: {len(items)} (기대: 29)")

    # 2) RAG 파이프라인 생성
    #    필요하면 config를 조정 (k 값 등)
    cfg = RAGConfig(
        k_law=5,
        k_rule=5,
        k_case=3,
        search_multiplier=2,
        enable_rerank=True,  # cohere 키가 없으면 rag_module 내부에서 에러 날 수 있어 False로 바꿔도 됨
    )
    pipeline = create_pipeline(config=cfg)

    # 3) RAG 실행 결과를 모아서 eval dataset 구성
    eval_ds = build_eval_dataset(pipeline, items, skip_normalization=False)

    # 4) RAGAS 평가 (법률 RAG 기준 “진단력 좋은” 기본 4종)
    result = evaluate(
        eval_ds,
        metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
    )

    print("\n====================")
    print("📌 RAGAS 전체 요약")
    print("====================")
    print(result)

    # 5) 문항별 점수 저장
    df = result.to_pandas()
    out_csv = "/mnt/data/ragas_result_by_question.csv"
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ 문항별 결과 CSV 저장: {out_csv}")

    # 6) (선택) 커스텀 총점 예시: Retrieval/Generation 분리 가중합
    #    - R = 0.6*context_recall + 0.4*context_precision
    #    - G = 0.6*faithfulness + 0.4*answer_relevancy
    #    - Total = 0.55*R + 0.45*G
    if all(c in df.columns for c in ["context_recall", "context_precision", "faithfulness", "answer_relevancy"]):
        df["R"] = 0.6 * df["context_recall"] + 0.4 * df["context_precision"]
        df["G"] = 0.6 * df["faithfulness"] + 0.4 * df["answer_relevancy"]
        df["Total"] = 0.55 * df["R"] + 0.45 * df["G"]
        out_csv2 = "/mnt/data/ragas_result_with_total.csv"
        df.to_csv(out_csv2, index=False, encoding="utf-8-sig")
        print(f"✅ 커스텀 총점 포함 CSV 저장: {out_csv2}")


if __name__ == "__main__":
    main()


C:\Users\Admin\AppData\Local\Temp\ipykernel_16360\2270681250.py:29: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
C:\Users\Admin\AppData\Local\Temp\ipykernel_16360\2270681250.py:29: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
C:\Users\Admin\AppData\Local\Temp\ipykernel_16360\2270681250.py:29: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: f

✅ 파싱된 문항 수: 29 (기대: 29)


2026-01-28 16:33:52,109 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-01-28 16:33:52,112 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?
2026-01-28 16:33:52,113 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?'
2026-01-28 16:33:52,633 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:33:54,900 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:33:57,247 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:33:59,923 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2026-01-28 16:33:59,929 - chatbot_app.modules.rag_module - INFO - 📌 Rerank selected=13 (threshold=0.2)
2026-01-28 16:34:00,599 - httpx 

2026-01-28 16:35:23,699 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-28 16:35:24,579 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-01-28 16:35:24,582 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 차임(증액)을 6개월 전에 했는데 또 차임증액을 요구해요. 따라야 하나요?
2026-01-28 16:35:24,583 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='차임(증액)을 6개월 전에 했는데 또 차임증액을 요구해요. 따라야 하나요?'
2026-01-28 16:35:25,280 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:35:26,071 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:35:27,188 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:35:28,340 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2026-01-28 16:35:28,35

2026-01-28 16:36:32,301 - chatbot_app.modules.rag_module - INFO - 🤖 답변 생성 중...
2026-01-28 16:36:40,342 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-28 16:36:40,984 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-01-28 16:36:41,000 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 확정일자(확정일자)를 받았으나, 그보다 먼저 설정된 근저당권(근저당권)이 존재합니다. 경매절차(경매)가 진행될 경우, 제 임대차보증금(보증금)이 해당 근저당권보다 우선변제권(우선변제권)에 따라 배당받을 수 있나요?
2026-01-28 16:36:41,000 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='확정일자(확정일자)를 받았으나, 그보다 먼저 설정된 근저당권(근저당권)이 존재합니다. 경매절차(경매)가 진행될 경우, 제 임대차보증금(보증금)이 해당 근저당권보다 우선변제권(우선변제권)에 따라 배당받을 수 있나요?'
2026-01-28 16:36:41,590 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:36:42,520 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-01-28 16:36:43

...

2026-01-28 17:26:16,324 - ragas.executor - ERROR - Exception raised in Job[52]: InstructorRetryException(<failed_attempts>

<generation number="1">
<exception>
    The output is incomplete due to a max_tokens length limit.
</exception>
<completion>
    ChatCompletion(id='chatcmpl-D2uzihzRU7vO0MlkZnaRzseX7L8p9', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "statements": [\n    {\n      "statement": "국세 체납으로 인한 가압류가 우선변제권을 가집니다.",\n      "reason": "The context discusses the priority of claims under the Housing Lease Protection Act, but does not explicitly state that tax arrears lead to priority over other claims.",\n      "verdict": 0\n    },\n    {\n      "statement": "경매 절차에 대한 정보 확인이 필요합니다.",\n      "reason": "The context implies the need for information regarding the auction process, making this statement reasonable.",\n      "verdict": 1\n    },\n    {\n      "statement": "경매 일정 및 진행 상황을 확인해야 합니다.",\n      "reason": "The

2026-01-28 17:26:16,326 - ragas.executor - ERROR - Exception raised in Job[53]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
2026-01-28 17:26:16,328 - ragas.executor - ERROR - Exception raised in Job[55]: InstructorRetryException(<failed_attempts>

<generation number="1">
<exception>
    The output is incomplete due to a max_tokens length limit.
</exception>
<completion>
    ChatCompletion(id='chatcmpl-D2v2R1eddTzyfWjHTbII2XJGct9Do', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "classifications": [\n        {\n            "statement": "상황에 따라 달라지며,",\n            "reason": "This statement does not provide specific information related to the context provided.",\n            "attributed": 0\n        },\n        {\n            "statement": "항상 임차인의 보증금이 먼저 보호되는 것은 아닙니다.",\n            "reason": "This statement reflects a general principle that is not explicitly mentioned in the context.",\n       

2026-01-28 17:26:16,330 - ragas.executor - ERROR - Exception raised in Job[56]: InstructorRetryException(<failed_attempts>

<generation number="1">
<exception>
    The output is incomplete due to a max_tokens length limit.
</exception>
<completion>
    ChatCompletion(id='chatcmpl-D2v4dNCu276mVXpUM4jUZqLs0JHCC', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "statements": [\n    {\n      "statement": "보증금 반환 소송이 유리할 수 있는 경우가 있습니다.",\n      "reason": "The context discusses various aspects of disputes related to deposit returns, implying that there are situations where a lawsuit could be favorable.",\n      "verdict": 1\n    },\n    {\n      "statement": "계약서 및 관련 서류를 준비해야 합니다.",\n      "reason": "The context mentions the importance of documentation in legal proceedings, indicating that preparing contracts and related documents is necessary.",\n      "verdict": 1\n    },\n    {\n      "statement": "임대인과의 대화 기록을 정리해야 합니다.",\n    

2026-01-28 17:26:16,330 - ragas.executor - ERROR - Exception raised in Job[57]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
2026-01-28 17:26:16,334 - ragas.executor - ERROR - Exception raised in Job[60]: InstructorRetryException(<failed_attempts>

<generation number="1">
<exception>
    The output is incomplete due to a max_tokens length limit.
</exception>
<completion>
    ChatCompletion(id='chatcmpl-D2v7XZP5VTF2Fe06EUBEY8gcTnTM5', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "statements": [\n    {\n      "statement": "계약 해지 시 위약금 조항이 있다면, 계약 해지에 따른 손해배상 책임이 발생할 수 있다.",\n      "reason": "계약 해지 시 위약금 조항이 존재하면 손해배상 책임이 발생할 수 있다는 것은 일반적인 법리로, 문맥에서 유추할 수 있다.",\n      "verdict": 1\n    },\n    {\n      "statement": "계약서의 위약금 조항을 정확히 확인해야 한다.",\n      "reason": "계약서의 위약금 조항을 확인하는 것은 계약 해지와 관련된 중요한 절차로, 문맥에서 유추할 수 있다.",\n      "verdict": 1\n    },\n    {\n      "statement": "계약 해지 사유가 정당한지 검토해야 한다.",\

2026-01-28 17:26:16,334 - ragas.executor - ERROR - Exception raised in Job[61]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
